In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime as dt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

In [6]:
# Load data for specific company
company = 'FB'

start = dt(2012, 1, 1)
end = dt(2021, 8, 1)

data = pdr.DataReader(company, 'yahoo', start, end)

In [7]:
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-05-18,45.000000,38.000000,42.049999,38.230000,573576400,38.230000
2012-05-21,36.660000,33.000000,36.529999,34.029999,168192700,34.029999
2012-05-22,33.590000,30.940001,32.610001,31.000000,101786600,31.000000
2012-05-23,32.500000,31.360001,31.370001,32.000000,73600000,32.000000
2012-05-24,33.209999,31.770000,32.950001,33.029999,50237200,33.029999
...,...,...,...,...,...,...
2021-07-26,374.440002,368.220001,369.579987,372.459991,14925200,372.459991
2021-07-27,373.149994,364.549988,371.910004,367.809998,15705400,367.809998
2021-07-28,377.549988,366.929993,374.559998,373.279999,29676900,373.279999
